In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('gym_data.csv')

In [3]:
data.drop(columns = ['Unnamed: 25','User'],inplace = True)

In [4]:
data['Category 11'][2712]= 2

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [5]:
data['Category 11'] = pd.to_numeric(data['Category 11'])

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5456 entries, 0 to 5455
Data columns (total 24 columns):
Category 1     5456 non-null float64
Category 2     5456 non-null float64
Category 3     5456 non-null float64
Category 4     5456 non-null float64
Category 5     5456 non-null float64
Category 6     5456 non-null float64
Category 7     5456 non-null float64
Category 8     5456 non-null float64
Category 9     5456 non-null float64
Category 10    5456 non-null float64
Category 11    5456 non-null float64
Category 12    5455 non-null float64
Category 13    5456 non-null float64
Category 14    5456 non-null float64
Category 15    5456 non-null float64
Category 16    5456 non-null float64
Category 17    5456 non-null float64
Category 18    5456 non-null float64
Category 19    5456 non-null float64
Category 20    5456 non-null float64
Category 21    5456 non-null float64
Category 22    5456 non-null float64
Category 23    5456 non-null float64
Category 24    5455 non-null float64
dtype

In [7]:
data['averge'] =np.round((data.sum(axis = 1))/(len(data.columns)),1)

### By considering  the mean of all categories as the satisfaction level for joining or taking the gym membership we are preparing the model based on this condition

In [8]:
data['membership'] = data.averge.map(lambda d: "Yes" if d > data.averge.mean() else "No")

### Base line model without pipeline and any parameter tuning

In [9]:
from sklearn.model_selection import train_test_split

In [10]:

from sklearn.preprocessing import LabelEncoder

In [11]:
data['membership_data'] = LabelEncoder().fit_transform(target_data)

NameError: name 'target_data' is not defined

In [ ]:
data.columns

In [ ]:
feature_data =  data.drop(columns = ['membership','membership_data','averge'])

In [ ]:
feature_data.fillna(data.averge.mean(),inplace = True)

In [ ]:
target_data  = data.membership_data

In [ ]:
trainX, testX, trainY, testY = train_test_split(feature_data, target_data)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [ ]:
feature_data.info()

In [ ]:
feature_data.isna().sum()

In [ ]:
names = ['linear_model','logistic_model','Decision_tree','Naive_bayes','randomforest','xgbclassifier']
models =  [LinearRegression(),LogisticRegression(),DecisionTreeClassifier(),GaussianNB(),RandomForestClassifier(),XGBClassifier()]
for name, estimator in zip(names, models):
    estimator.fit(trainX,trainY)
    print(estimator.score(testX,testY))

### Feature_selection 

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import f_classif, SelectKBest

In [ ]:
float_data = feature_data.select_dtypes(include = ['float'])

In [ ]:
int_data = feature_data.select_dtypes(include=['int64'])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
float_pipeline = make_pipeline(MinMaxScaler())
int_pipeline = make_pipeline(MinMaxScaler())

In [ ]:
from sklearn.compose import make_column_transformer

In [ ]:
preprocessor = make_column_transformer(
(int_pipeline, int_data.columns),(float_pipeline, float_data.columns), remainder = 'passthrough'
)

In [ ]:
from sklearn.preprocessing import StandardScaler


In [ ]:
pipeline = make_pipeline(StandardScaler(), SelectKBest(k = 10, score_func = f_classif), RandomForestClassifier(n_estimators = 100))

In [ ]:
pipeline.fit(trainX, trainY)

In [ ]:
pipeline.score(testX, testY)

In [ ]:
names = ['linear_model','logistic_model','Decision_tree','Naive_bayes','randomforest','xgbclassifier']
models =  [LinearRegression(),LogisticRegression(),DecisionTreeClassifier(),GaussianNB(),RandomForestClassifier(n_estimators= 100),XGBClassifier()]
for name, estimator in zip(names, models):
    pipeline = make_pipeline(StandardScaler(), SelectKBest(k = 10, score_func = f_classif), estimator)
    pipeline.fit(trainX,trainY)
    print(name,pipeline.score(testX,testY))